### CNN Model for the Phishing Websites group project

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msn
from scipy.io import arff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras import layers, models

2023-12-06 23:48:46.586603: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data = arff.loadarff('Training Dataset.arff')
df = pd.DataFrame(data[0])
df = df.apply(lambda col: col.str.decode('utf-8').str.strip("'"))

df.head()

,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,...,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
0,-1,1,1,1,-1,-1,-1,-1,-1,1,...,1,1,-1,-1,-1,-1,1,1,-1,-1
1,1,1,1,1,1,-1,0,1,-1,1,...,1,1,-1,-1,0,-1,1,1,1,-1
2,1,0,1,1,1,-1,-1,-1,-1,1,...,1,1,1,-1,1,-1,1,0,-1,-1
3,1,0,1,1,1,-1,-1,-1,1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
4,1,0,-1,1,1,-1,1,1,-1,1,...,-1,1,-1,-1,0,-1,1,1,1,1


In [3]:
X = df.drop("Result", axis=1)
Y = df['Result']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

X_train = X_train.astype(float)
X_test = X_test.astype(float)
Y_train = Y_train.astype(float)
Y_test = Y_test.astype(float)

In [4]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape the input data to be 3D 
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

model = models.Sequential()

model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))

model.add(layers.MaxPooling1D(pool_size=2))

model.add(layers.Flatten())

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(1, activation='sigmoid'))  

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
111/111 [==============================] - 1s 4ms/step - loss: -7.2271 - accuracy: 0.1608 - val_loss: -28.7278 - val_accuracy: 0.2024
Epoch 2/10
111/111 [==============================] - 0s 2ms/step - loss: -92.2677 - accuracy: 0.1972 - val_loss: -212.4204 - val_accuracy: 0.1792
Epoch 3/10
111/111 [==============================] - 0s 2ms/step - loss: -416.6284 - accuracy: 0.1886 - val_loss: -769.1656 - val_accuracy: 0.1973
Epoch 4/10
111/111 [==============================] - 0s 2ms/step - loss: -1195.1327 - accuracy: 0.1907 - val_loss: -1906.7886 - val_accuracy: 0.1945
Epoch 5/10
111/111 [==============================] - 0s 2ms/step - loss: -2598.1189 - accuracy: 0.1927 - val_loss: -3790.8274 - val_accuracy: 0.1922
Epoch 6/10
111/111 [==============================] - 0s 2ms/step - loss: -4779.1016 - accuracy: 0.1915 - val_loss: -6576.9038 - val_accuracy: 0.1899
Epoch 7/10
111/111 [==============================] - 0s 2ms/step - loss: -7851.1543 - accuracy: 0.1900 - val_

In [5]:
predictions = model.predict(X_test)
predictions_binary = np.round(predictions)
accuracy = accuracy_score(Y_test, predictions_binary)
print(f"Accuracy: {accuracy}")


70/70 [==============================] - 0s 978us/step
Accuracy: 0.18317503392130258
